In [1]:
green_areas = r'D:\osm2lulc2023\rule3_4\rule34.gdb\result_evu_v2'

green_fid = 'ffid'

osm_lines = r'D:\osm2lulc2023\rule3_4\rule34.gdb\osm\osmlines'

outfeat = r'D:\osm2lulc2023\rule3_4\rule34.gdb\result_evu_v3'

tmpws = r'D:\osm2lulc2023\rule3_4\rule34.gdb\corre'

In [2]:
import arcpy
import os

from gesri.dp import copy_feat
from gesri.rd.shp import shp_to_lyr
from gesri.gp.ovl import intersection
from gesri.gp.gen import dissolve
from gesri.tbl.joins import join_table
from gesri.tbl.attr import geom_attr_to_shp

In [3]:
srs = "PROJCS[\"ETRS_1989_Portugal_TM06\",GEOGCS[\"GCS_ETRS_1989\",DATUM[\"D_ETRS_1989\",SPHEROID[\"GRS_1980\",6378137.0,298.257222101]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"False_Easting\",0.0],PARAMETER[\"False_Northing\",0.0],PARAMETER[\"Central_Meridian\",-8.133108333333334],PARAMETER[\"Scale_Factor\",1.0],PARAMETER[\"Latitude_Of_Origin\",39.66825833333333],UNIT[\"Meter\",1.0]]"

In [4]:
arcpy.env.overwriteOutput = True

In [5]:
cp_green = copy_feat(green_areas, outfeat)

glyr = shp_to_lyr(cp_green)

llyr = shp_to_lyr(osm_lines)

In [6]:
# Intersect

ointersect = os.path.join(tmpws, 'i_green_osmlines')
ilyr = intersection([glyr, llyr], ointersect)

In [7]:
# Dissolve

odiss = os.path.join(tmpws, 'green_lines_diss')
disslyr = dissolve(ilyr, odiss, green_fid)

# Get length
disslyr = geom_attr_to_shp(
    disslyr, "cline",
    geom_attr="LENGTH", srs=srs,
    lenunit="METERS"
)

In [8]:
# Join

green_lyr = join_table(
    cp_green, disslyr,
    green_fid, green_fid,
    cols=["cline"]
)